In [47]:
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
from datetime import date
import os

In [48]:
DARK_BLUE = '#3557A9'
DARK_GREEN = '#50974F'
DARK_RED = '#D43F2C'
DARK_YELLOW = '#F8D65A'

BLUE = '#627EC3'
GREEN = '#7ABA79'
RED = '#E36656'
YELLOW = '#FFE480'

LIGHT_BLUE = '#8CA5E2'
LIGHT_GREEN = '#A3D9A3'
LIGHT_RED = '#F6887A'
LIGHT_YELLOW = '#FFEA9D'

In [49]:
proposals_path = 'C:\\Users\\leibr\\Documents\\Lappis\\conjuv\\data\\conjuv_propostas.csv'

df_sheets = pd.read_csv(proposals_path, sep=';', encoding='utf-8')

import pandas as pd

# Crie um filtro para selecionar os dados desejados
data = (df_sheets['participatory_space/url'] == 'http://brasilparticipativo.presidencia.gov.br/assemblies/confjuv4?assembly_slug=confjuv4') & \
         (~df_sheets['state'].isin(['rejected', 'withdrawn']))

# Aplique o filtro ao DataFrame
new_data = df_sheets[data]

report_path = 'relatorio_conjuv.md'

### Top 20 propostas por voto

In [50]:
# Classificar os dados por número de votos em ordem decrescente
new_data = new_data.sort_values(by='supports', ascending=False)

# Selecionar os Top 20 Programas
top_20_programas = new_data.head(20)

# Criar um novo DataFrame para a Tabela de Ranking
tabela_ranking = top_20_programas[['title/pt-BR', 'category/name/pt-BR', 'supports', 'comments']].copy()
tabela_ranking.columns = ['Propostas', 'Temas', 'Nº de Votos', 'Nº de Comentários']

# Crie a tabela formatada em Markdown
tabela_ranking_markdown = tabulate(tabela_ranking, headers='keys', tablefmt='pipe')

### Ranking por tema

In [51]:
# Agrupe os dados por 'category/name/pt-BR' e agregue as contagens
temas_df = new_data.groupby('category/name/pt-BR').agg({
    'title/pt-BR': 'nunique',  # Contagem de propostas únicas
    'supports': 'sum',         # Soma dos votos
    'comments': 'sum'         # Soma dos comentários
}).reset_index()

# Renomeie as colunas
temas_df.columns = ['Temas', 'Nº de Propostas', 'Nº de Votos', 'Nº de Comentários']

# Crie a tabela formatada em Markdown
temas_df_markdown = tabulate(temas_df, headers='keys', tablefmt='pipe')

### Ranking Tópicos

In [52]:
import pandas as pd

topics_path = 'C:\\Users\\leibr\\Documents\\Lappis\\conjuv\\data\\topics.csv'
# Carregar o arquivo topics.csv em latin-1
topics_df = pd.read_csv(topics_path, sep=';', encoding='utf-8')

# Selecionar apenas as colunas 'topic_1', 'topic_2' e 'topic_3'
topics_df = topics_df[['Tópico 1', 'Tópico 2', 'Tópico 3']]


In [53]:
# Inicialize listas vazias para os tópicos mais citados de cada coluna
top_topic_1 = []
top_topic_2 = []
top_topic_3 = []

# Função para obter os tópicos mais citados em uma coluna e separá-los por linhas
def get_top_topics(column):
    all_topics = column.str.extract('(.+?) \(')[0]
    topic_counts = all_topics.value_counts()
    top_topics = topic_counts.head(20).index.tolist()
    return '\n'.join(top_topics)

# Obtenha os tópicos mais citados em cada coluna e armazene-os nas listas
top_topic_1.append(get_top_topics(topics_df['Tópico 1']))
top_topic_2.append(get_top_topics(topics_df['Tópico 2']))
top_topic_3.append(get_top_topics(topics_df['Tópico 3']))

# Crie um novo DataFrame com as listas de tópicos mais citados
top_topics_df = pd.DataFrame({
    'Tópico 1': top_topic_1,
    'Tópico 2': top_topic_2,
    'Tópico 3': top_topic_3
})


# Para exibir a tabela formatada em Markdown, você pode fazer:
top_topics_df = tabulate(top_topics_df, headers='keys', tablefmt='pipe')

#markdown_table = top_topics_df.to_markdown(index=False)

#print(top_topics_df)

In [54]:
total_propostas = len(new_data)
propostas_acima_de_30_votos = len(new_data[new_data['supports'] > 30])
propostas_abaixo_de_10_votos = len(new_data[new_data['supports'] < 10])
porcentagem_abaixo_de_10_votos = (propostas_abaixo_de_10_votos / total_propostas) * 100
menor_votos_entre_propostas = new_data['supports'].min()

In [55]:
# Crie o título com a data atual
titulo = f"<center>Relatório Conjuv ({date.today().strftime('%d/%m/%Y')})</center>"

# Caminho para o arquivo Markdown
report_path = 'relatorio_conjuv.md'

# Crie o arquivo Markdown com o título e a imagem
with open(report_path, 'w', encoding='utf-8') as report_file:
    report_file.write(f"# {titulo}\n\n")
    report_file.write("</p>\n\n")

# Abra o arquivo novamente para adicionar o sumário
with open(report_path, 'a', encoding='utf-8') as report_file:
    report_file.write("\n# Sumário:\n")
    report_file.write("- [TOP 20 Propostas por Voto no Geral](#top-20-propostas-por-voto-no-geral)\n")
    report_file.write("- [Análise Qualitativa das Propostas](#análise-qualitativa-das-propostas)\n")

    report_file.write("- [Ranking por Tópicos](#ranking-por-tópicos)\n")
    
# Abra o arquivo Markdown para adicionar as tabelas
with open(report_path, 'a', encoding='utf-8') as report_file:
    report_file.write(f"## *TOP 20 Propostas por Voto no Geral*\n{tabela_ranking_markdown}\n\n")
    report_file.write(f"## *Análise Qualitativa das Propostas* \n Os temas com o maior número de propostas são: \n{temas_df_markdown}\n")
    report_file.write(f"- De {total_propostas} propostas atuais, apenas {propostas_acima_de_30_votos} delas apresentam votação acima de 30;\n")
    report_file.write(f"- {propostas_abaixo_de_10_votos} propostas estão com menos de 10 votos ({porcentagem_abaixo_de_10_votos:.2f}% do total de propostas);\n")
    report_file.write(f"- Todos os programas apresentam votação acima de {menor_votos_entre_propostas};\n")

    
    report_file.write(f"## *Ranking por Tópicos* \n{top_topics_df}\n")

